In [131]:
import os
import pandas as pd
TOP_DIR = '../../..'
RAW_DATA_DIR = f'{TOP_DIR}/data-raw'
DATA_DIR = f'{TOP_DIR}/data'

In [132]:
rental_prices = pd.read_csv(f'{DATA_DIR}/rental-prices/rental-prices.csv')
wages = pd.read_csv(f'{DATA_DIR}/ashe/weekly-earnings.csv')

In [133]:
rental_prices = rental_prices[(rental_prices.date == max(rental_prices.date)) 
                              &(rental_prices.variable_code == 'Lower quartile')
                              &(rental_prices.property_name == 'bed2')]
rental_prices = rental_prices[['geography_code', 'value']].set_index('geography_code')
rental_prices.rename(columns={'value': 'lower_quartile_rent'}, inplace=True)
rental_prices

,lower_quartile_rent
geography_code,
E92000001,625
E12000001,430
E06000047,395
E06000005,425
E06000001,400
...,...
E10000027,630
E07000187,695
E07000188,575


In [134]:
wages = wages[
    (wages.date == max(wages.date)) 
    &(wages.variable_name == 'median_weekly_wage')]
wages = wages[['geography_code', 'value']].set_index('geography_code')
wages.rename(columns={'value': 'median_weekly_wage'}, inplace=True)
wages

,median_weekly_wage
geography_code,
E12000001,490.6
E06000005,482.2
E06000001,498.3
E06000002,454.5
E06000003,440.7
...,...
E08000032,476.9
E08000033,494.4
E08000034,513.5


In [135]:
merged = wages.merge(rental_prices, left_index=True, right_index=True)
print(merged.columns.to_list())
merged['lq_rent_to_median_earnings'] = merged['lower_quartile_rent'] / (4*merged['median_weekly_wage'])
merged

['median_weekly_wage', 'lower_quartile_rent']


,median_weekly_wage,lower_quartile_rent,lq_rent_to_median_earnings
geography_code,,,
E12000001,490.6,430,0.219119
E06000005,482.2,425,0.220344
E06000001,498.3,400,0.200682
E06000002,454.5,433,0.238174
E06000003,440.7,425,0.241094
...,...,...,...
E08000032,476.9,475,0.249004
E08000033,494.4,475,0.240190
E08000034,513.5,470,0.228822


In [136]:
merged.melt(value_vars='lq_rent_to_median_earnings', ignore_index=False).to_csv(f'{DATA_DIR}/rental-prices/rent-earning-ratio.csv')